In [1]:
import torch
import torchvision

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = True
torch.manual_seed(random_seed)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [4]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [5]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [6]:
example_data.shape

torch.Size([1000, 1, 28, 28])

In [7]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig.show()

In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [10]:
network = Net()
network = network.to(device)
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [11]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [12]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device)
    target = target.to(device)
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './results/model.pth')
      torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [13]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data = data.to(device)
      target = target.to(device)
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [14]:
# test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307344
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.293903
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.268074
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.283568
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.223637
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.229984
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.101499
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.071804
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.065552
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.831051
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.911197
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.670233
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.648533
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.865548
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.616167
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.411429
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.302792
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.340002
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.151308
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.359005
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.2012, Accuracy: 9405/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.500734
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.464070
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.451652
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.407594
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.469546
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.645535
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.472790
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.338747
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.456496
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.350108
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.646125
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.525309
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.361591
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.387272
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.364907
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.354882
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.260317
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.462571
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.552020
Train Epoch: 2 [12160